In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import json
import malaya
from tqdm import tqdm
from unidecode import unidecode

/home/husein/dev/malaya/malaya/tokenizer.py:208: FutureWarning: Possible nested set at position 3386
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:208: FutureWarning: Possible nested set at position 3904
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [3]:
import re

def cleaning(splitted, split = False):
    if split:
        splitted = malaya.text.function.split_into_sentences(splitted)
    if splitted[0][0] == '-':
        splitted[0] = splitted[0].replace('- ','')
    points = [f'{no + 1}. {s}' for no, s in enumerate(splitted)]
    points = ' '.join(points)
    return points

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('\r', ' ')).strip()

In [4]:
months = {
    'january',
    'jan',
    'januari',
    'february',
    'feb',
    'februari',
    'march',
    'mac',
    'april',
    'apr',
    'may',
    'mei',
    'june',
    'jun',
    'july',
    'julai',
    'august',
    'ogos',
    'aug',
    'september',
    'sep',
    'october',
    'oktober',
    'oct',
    'november',
    'nov',
    'december',
    'disember',
    'dec',
    'utusan',
    'malaysiakini',
    'astroawani',
    'bernama',
    'com',
}

malaysian_news = {
    'kosmo',
    'hmetro',
    'malaymail',
    'projekmm',
    'bharian',
    'utusan',
    'astroawani',
    'themalaysianinsight',
    'malaysiakini',
    'bernama',
    'abdullahjones',
    'daily',
    'awani',
    'astro',
}

def remove_colon(string):
    string = re.sub(r'[ ]+', ' ', string).strip()
    if string.strip()[0] == ':':
        string = string[1:]
    return re.sub(r'[ ]+', ' ', string).strip()

def remove_first_dash_month(string):
    string = unidecode(string)
    if 0 < string.find('-') < 50:
        sub = string[:string.find('-')]
        if len(set(sub.lower().split()) & months):
            string = string[string.find('-') + 1:]
    return re.sub(r'[ ]+', ' ', string).strip()

def remove_last_dash(string):
    rfind = string.rfind('-')
    if rfind > 0 and len(string[rfind:]) < 50:
        if any([m in string.lower() for m in malaysian_news]):
            string = string[:rfind]
    return re.sub(r'[ ]+', ' ', string).strip()

def remove_last_dash_dash(string):
    rfind = string.rfind('--')
    if rfind > 0 and len(string[rfind:]) < 50:
        if any([m in string.lower() for m in malaysian_news]):
            string = string[:rfind]
    return re.sub(r'[ ]+', ' ', string).strip()
    
def clean_news(string, filter_news = True):
    splitted = malaya.text.function.split_into_sentences(string, minimum_length = 10)
    if filter_news:
        filtered = []
        for s in splitted:
            if not any([m in s.lower() for m in malaysian_news]):
                filtered.append(s)
    else:
        filtered = splitted
    
    s = ' '.join(filtered)
    s = remove_colon(s)
    s = remove_first_dash_month(s)
    s = remove_last_dash(s)
    s = remove_last_dash_dash(s)
    return s

In [5]:
!ls -lh news.json

-rw-r--r-- 1 husein husein 541M Dis   1 18:14 news.json


In [6]:
!ls -lh news-astroawani.json

-rw-r--r-- 1 husein husein 414M Dis   4 16:47 news-astroawani.json


In [7]:
!ls -lh article.json

-rw-r--r-- 1 husein husein 376M Dis   1 22:27 article.json


In [8]:
!ls -lh karangan.json

-rw-r--r-- 1 husein husein 15M Dis   1 20:57 karangan.json


In [9]:
!ls -lh keywords-headline.json

-rw-r--r-- 1 husein husein 128M Dis   1 22:19 keywords-headline.json


In [10]:
!ls -lh keywords-headline-astroawani.json

-rw-r--r-- 1 husein husein 100M Dis   4 16:58 keywords-headline-astroawani.json


In [11]:
!ls -lh product-description.json

-rw-r--r-- 1 husein husein 304M Dis   1 21:37 product-description.json


In [12]:
train = open('train.json', 'w')

In [13]:
with open('news.json') as fopen:
    data = json.load(fopen)
    
for d in tqdm(data):
    
    if len(set(d[0].split())) < 20:
        continue
        
    try:
        src = cleaning(d[0], split = True)
        tgt = clean_news(simple_cleaning(d[1]), filter_news = False)

        d = {"translation": {"src": src, "tgt": tgt, 'prefix': 'surat-khabar: '}}
        train.write(f'{json.dumps(d)}\n')
    except Exception as e:
        print(e)

  7%|██▍                               | 16019/219612 [00:11<02:19, 1454.55it/s]

list index out of range


 28%|█████████▎                        | 60533/219612 [00:44<02:10, 1219.77it/s]

list index out of range


 39%|█████████████▋                     | 86230/219612 [01:08<02:24, 922.90it/s]

...

 88%|█████████████████████████████    | 193560/219612 [02:45<00:21, 1191.51it/s]

list index out of range


 90%|█████████████████████████████▋   | 197677/219612 [02:49<00:21, 1012.82it/s]

string index out of range
string index out of range


100%|█████████████████████████████████| 219612/219612 [03:08<00:00, 1167.16it/s]


In [14]:
!tail -n 1 train.json

{"translation": {"src": "1. Kazuki Yamamoto, 29, dari wilayah Okayama mencipta pelbagai haiwan kecil yang kelihatan realistik ketika bekerja di sebuah restoran. 2. Hasil seninya mula mendapat perhatian di seluruh dunia.", "tgt": "Seorang barista di Jepun mencipta hasil seni menarik dan comel hanya menggunakan buih susu dalam latte dihidangkannya. Kazuki Yamamoto, 29, dari wilayah Okayama mencipta pelbagai haiwan kecil yang nampak realistik ketika bekerja di sebuah restoran, sebelum hasil seninya itu mula mendapat perhatian di seluruh dunia. Kerja tangannya yang paling popular seekor kucing yang cuba masuk dari satu cawan ke cawan lain untuk menangkap ikan. Dia yang kini pengurus sebuah kafe di Harajyuku juga membuat buih berbentuk arnab dan kucing yang lain, dianggap pelanggannya sebagai 'terlalu comel sehingga tidak tergamak untuk meminumnya. ' \"Gambar kucing menangkap ikan menjadi popular selepas tersebar di Internet sehingga saya mendapat tawaran untuk menghadiri pelbagai acara dan

In [15]:
with open('news-astroawani.json') as fopen:
    data = json.load(fopen)
    
for d in tqdm(data):
    
    if len(set(d[0].split())) < 20:
        continue
        
    try:
        src = cleaning(d[0], split = True)
        tgt = clean_news(simple_cleaning(d[1]), filter_news = False)

        d = {"translation": {"src": src, "tgt": tgt, 'prefix': 'surat-khabar: '}}
        train.write(f'{json.dumps(d)}\n')
    except Exception as e:
        print(e)

 31%|██████████▌                       | 55326/178407 [00:42<01:43, 1187.78it/s]

list index out of range


 36%|████████████▏                     | 63758/178407 [00:49<01:28, 1292.63it/s]

list index out of range


 39%|█████████████▍                    | 70381/178407 [00:54<01:17, 1391.14it/s]

...

 52%|█████████████████▊                | 93387/178407 [01:12<01:09, 1220.76it/s]

list index out of range


 60%|███████████████████▊             | 107355/178407 [01:24<00:52, 1364.80it/s]

list index out of range


100%|█████████████████████████████████| 178407/178407 [02:23<00:00, 1242.52it/s]


In [16]:
!tail -n 1 train.json

{"translation": {"src": "1. Sekumpulan remaja Australia mula dikenali apabila memuatnaik video klip \"HarlemShake v1\" ke laman YouTube pada 2 Februari. 2. Sehingga kini, video tersebut telah ditonton lebih 12 juta orang dan angkanya terus meningkat.", "tgt": "Sekumpulan belia Australia telah mencetuskan mania baru di Internet apabila menghasilkan sebuah gerak tari yang digelar \"HarlemShake\", iaitu tarian gerakan pinggul yang sehingga kini telah membuatkan ribuan pengguna Internet meniru aksi itu. Lima remaja itu yang berasal dari Sunshine Coast, Australia mula dikenali apabila memuatnaik rakaman video klip \"The Harlem Shake v1(TSCS original)\" (video di bawah) yang berdurasi 31 saat ke laman YouTube pada 2 Februari. Sehingga kini ia telah ditonton lebih 12 juta orang dan angkanya terus meningkat. Tarian gerakan pinggul itu bermula dengan kemunculan seorang lelaki bertopi keledar yang menari mengikut rentak lagu \"HarlemShake\" yang dihasilkan oleh artis meningkat naik dari Brooklyn

In [17]:
with open('article.json') as fopen:
    data = json.load(fopen)
    
for i in tqdm(range(len(data['before']))):
        
    try:
        src = cleaning(data['before'][i])
        tgt = simple_cleaning(data['after'][i])

        d = {"translation": {"src": src, "tgt": tgt, 'prefix': 'artikel: '}}
        train.write(f'{json.dumps(d)}\n')
    except Exception as e:
        print(e)

100%|████████████████████████████████| 130390/130390 [00:05<00:00, 22782.70it/s]


In [18]:
!tail -n 1 train.json

{"translation": {"src": "1. Majlis Evangelical untuk Akauntabiliti Kewangan (ECFA) adalah persatuan piawaian kewangan Amerika yang mewakili organisasi dan gereja Kristian evangelis, yang memenuhi syarat untuk status bebas cukai, bukan untung dan menerima sumbangan yang boleh ditolak cukai. 2. Ditubuhkan pada tahun 1979, ECFA mengakreditasi lebih dari 2,200 organisasi anggota yang telah menunjukkan kepatuhan terhadap standard kewangannya. 3. Sehingga 2015, pendapatan tahunan kolektif organisasi anggota ECFA dilaporkan hampir $ 25 bilion.", "tgt": "Pada tahun 1979, ECFA diasaskan oleh Billy Graham Evangelistic Association dan cabang US World Vision (World Vision International bukan anggota ECFA). ECFA ditubuhkan dengan penubuhan tujuh standard pertanggungjawaban yang merangkumi tadbir urus lembaga, syarat untuk penyata kewangan yang diaudit, keperluan untuk pendedahan orang ramai mengenai penyata kewangan yang diaudit, penghindaran konflik kepentingan, dan standard mengenai aktiviti peng

In [19]:
with open('karangan.json') as fopen:
    data = json.load(fopen)
    
for i in tqdm(range(len(data['before']))):
    try:
        src = cleaning(data['before'][i])
        tgt = simple_cleaning(data['after'][i])

        d = {"translation": {"src": src, "tgt": tgt, 'prefix': 'karangan: '}}
        train.write(f'{json.dumps(d)}\n')
    except Exception as e:
        print(e)

100%|████████████████████████████████████| 2533/2533 [00:00<00:00, 11805.29it/s]


In [20]:
!tail -n 1 train.json

{"translation": {"src": "1. selebriti dan orang-orang ternama negara seperti menteri dan tokoh korporat perlu menunjukkan tauladan dan menggalakkan masyarakat menggunakan laman sosial masing-masing 2. kempen secara meluas dan menggunakan sentimen kemasyarakatan pasti dapat menggalakkan orang ramai untuk melakukannya 3. Doktor pula memerlukan bekalan darah bagi menstabilkan bilangan darah pesakit 4. menderma adalah dapat membantu pesakit 5. apakah kebaikan dan langkah menggalakkan orang ramai menderma demi manfaat bersama 6. Keperluan dan permintaan terhadap darah meningkat dari tahun ke tahun hasil perkembangn industri kesihatan di dalam negara 7. Bagi menggalakkan orang ramai 8. orang ramai boleh dipengaruhi untuk menderma 9. hospital memerlukan sumbangan daripada pesakit 10. penting dalam merawat pesakit di hospital 11. dalam kesusahan dan boleh dianggap sebagai wira 12. Seperti ungkapan orang-orang tua 13. bukan sahaja memudahkan aktiviti malah dapat membudayakannya sebagai satu tan

In [21]:
with open('keywords-headline.json') as fopen:
    data = json.load(fopen)

for i in tqdm(range(len(data['before']))):
    try:
        src = cleaning(data['before'][i])
        tgt = clean_news(simple_cleaning(data['after'][i]), filter_news = False)

        d = {"translation": {"src": src, "tgt": tgt, 'prefix': 'tajuk-surat-khabar: '}}
        train.write(f'{json.dumps(d)}\n')
    except Exception as e:
        print(data['after'][i])
        print(e)
        break

100%|█████████████████████████████████| 155967/155967 [00:32<00:00, 4811.14it/s]


In [22]:
!tail -n 1 train.json

{"translation": {"src": "1. Pengurus Taman Hidupan Liar Sunway Lagoon 2. majikan menyediakan kursus penjagaan binatang 3. ramai menyayangi hidupan liar 4. Singapore Zoological Garden", "tgt": "Tidak lama kemudian majikan menyediakan kursus penjagaan binatang. Azman kini menjawat jawatan sebagai Pengurus Taman Hidupan Liar Sunway Lagoon yang baginya mempunyai tarikan tersendiri. \"Taman ini boleh dianggap sebagai zoo butik dan perkara yang menjadi keutamaan adalah untuk memberi pendidikan kepada orang ramai menyayangi hidupan liar serta mengenali binatang dengan lebih rapat. \"Anak yang sulung, Shakir kini bertugas di Singapore Zoological Garden dan mengkhusus dalam penjagaan burung, manakala yang kedua, Shabri (24 tahun) menjadi penjaga haiwan di Zoo Melaka,\" ujarnya.", "prefix": "tajuk-surat-khabar: "}}


In [23]:
with open('keywords-headline-astroawani.json') as fopen:
    data = json.load(fopen)

for i in tqdm(range(len(data['before']))):
    try:
        src = cleaning(data['before'][i])
        tgt = clean_news(simple_cleaning(data['after'][i]), filter_news = False)

        d = {"translation": {"src": src, "tgt": tgt, 'prefix': 'tajuk-surat-khabar: '}}
        train.write(f'{json.dumps(d)}\n')
    except Exception as e:
        print(data['after'][i])
        print(e)
        break

 94%|███████████████████████████████  | 114488/121822 [00:24<00:01, 4746.45it/s]

Maka dengan itu ramai yang tidak sabar melakukan aktiviti yang sebelum ini dilarang sama sekali termasuk beriadah. Tunggu sehingga keadaan terkawal dan tidak ramai orang.
list index out of range


In [24]:
!tail -n 1 train.json

{"translation": {"src": "1. versi bahasa Kabong versi 2. kandungan dihasilkan Pak Wo 3. Pak Wo Kabong hati 4. Terdapat versi bahasa Kabong 5. bahasa Kabong versi moden 6. bahasa Kabong Contohnya perkataan 7. dihasilkan Pak Wo berkait", "tgt": "Pak Wo Kabong semakin mendapat tempat di hati penduduk daerah kecil di Sarawak ini. \"Terdapat dua versi bahasa Kabong iaitu versi moden dan versi lama yang semakin hilang. TV Kabong kini memiliki lebih 25 ribu pengikut di laman Facebook dan setiap kandungan video yang dihasilkan Pak Wo berkait rapat mengenai isu semasa dan diolah menggunakan bahasa Kabong. \"Contohnya perkataan letik dah semakin kurang digunakan sebaliknya orang ramai memilih menggunakan perkataan lekat atau lekit,\" tambah Pak Wo.", "prefix": "tajuk-surat-khabar: "}}


In [25]:
with open('product-description.json') as fopen:
    data = json.load(fopen)
    
for i in tqdm(range(len(data['before']))):
    try:
        src = cleaning(data['before'][i])
        tgt = simple_cleaning(data['after'][i])

        d = {"translation": {"src": src, "tgt": tgt, 'prefix': 'penerangan-produk: '}}
        train.write(f'{json.dumps(d)}\n')
    except Exception as e:
        print(e)

100%|████████████████████████████████| 360944/360944 [00:05<00:00, 62695.70it/s]


In [26]:
!tail -n 1 train.json

{"translation": {"src": "1. permainan aksi platform scrolling sampingan 2. Nintendo Entertainment System", "tgt": "Amagon, yang dikenali di Jepun sebagai Totsuzen! Machoman (! lit. Tiba-tiba! Machoman?), adalah permainan aksi platform scrolling sampingan untuk Nintendo Entertainment System yang dikembangkan oleh Aicom.", "prefix": "penerangan-produk: "}}


In [27]:
train.close()

In [28]:
!shuf train.json > shuffled-train.json

In [29]:
!head -n 5000 shuffled-train.json > test.json

In [30]:
!wc -l train.json

1153570 train.json
